# Projeto #1 - Mineração de padrões frequentes

<p style='text-align: justify;'><font size=3.>Análise de acidentes nas rodovias federais brasileira entre 2007 e 2021.</font></p>

> <p style='text-align: justify;'><font size=2.85>Esse projeto utiliza a versão 0.17.0 do mlxtend.</font></p>

In [1]:
import mlxtend
print(mlxtend.__version__)

0.17.0


### Bibliotecas básicas e outros imports

In [2]:
import warnings
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

%run modules/text.py

### Carregamento dos dados

In [3]:
df = pd.read_csv('dataset/cleaned_acidentes_rodovias_brasileiras_2007_a_2021.csv', low_memory=False, encoding='utf-8')
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')
df.head(1)

Linhas: 1894736 | Coluna:  24


,id,dia_semana,uf,br,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,tipo_envolvido,estado_fisico,data,mes,ano,estacao_do_ano,regiao_pais,estacao,clima
0,83123214.0,quarta,SC,280,Rio negrinho,outras,saida de pista,ignorada,plena noite,decrescente,chuva,simples,curva,urbano,automovel,condutor,ignorada,2014-01-01,1,2014,verão,sul,3,1


### Remoção valores inválidos

In [4]:
df.drop(df[df['tipo_veiculo'] == 'nao identificado'].index, inplace=True)
df.drop(df[df['tipo_veiculo'] == 'nao informado'].index, inplace=True)
df.drop(df[df['classificacao_acidente'] == 'ignorada'].index, inplace=True)
df.drop(df[df['condicao_metereologica'] == 'ignorada'].index, inplace=True)
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')

Linhas: 1844479 | Coluna:  24


### Geração de features

#### Agrupamento tipos de acidentes

In [5]:
df['tipo_acidente'] = df['tipo_acidente'].apply(mapTiposDeAcidentes)

#### Agrupamento fase do dia

In [6]:
df['fase_dia'] = df['fase_dia'].map(mapFase)

#### Agrupamento dias da semana

In [7]:
df['dia_semana'] = df['dia_semana'].apply(mapDiasDaSemana)

#### Agrupamento condição meteorológica

In [8]:
df['condicao_clima'] = df['condicao_metereologica'].map(mapClima)

#### Agrupamento tamanho veículos

In [9]:
df['porte_veiculo'] = df['tipo_veiculo'].apply(mapTamanhoVeiculos)

### Seleção dos recursos para análise

In [10]:
cols = [
        'classificacao_acidente',
        'tipo_acidente', 
        'fase_dia',
        'condicao_clima', 
        'tracado_via',
        'porte_veiculo'
        ]
data = df[cols]
data.head()

,classificacao_acidente,tipo_acidente,fase_dia,condicao_clima,tracado_via,porte_veiculo
1,sem vitimas,colisao traseira,noite,tempo ruim,reta,pequeno porte
2,sem vitimas,atropelamento de animal,noite,tempo bom,reta,pequeno porte
3,sem vitimas,colisao transversal,noite,tempo bom,cruzamento,pequeno porte
4,com vitimas feridas,colisao frontal,noite,nublado,reta,pequeno porte
5,sem vitimas,colisao traseira,noite,nublado,reta,pequeno porte


### Agrupamento

#### 1º cluster: Sem vítimas

In [11]:
sem_vitimas = data.query('classificacao_acidente=="sem vitimas"')
sem_vitimas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {sem_vitimas.shape[0]} | Coluna:  {sem_vitimas.shape[1]}')

Linhas: 968151 | Coluna:  5


#### 2º cluster: Vítimas feridas



In [12]:
vitimas_feridas = data.query('classificacao_acidente=="com vitimas feridas"')
vitimas_feridas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_feridas.shape[0]} | Coluna:  {vitimas_feridas.shape[1]}')

Linhas: 797703 | Coluna:  5


#### 3º cluster: Vítimas fatais

In [13]:
vitimas_fatais = data.query('classificacao_acidente=="com vitimas fatais"')
vitimas_fatais.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_fatais.shape[0]} | Coluna:  {vitimas_fatais.shape[1]}')

Linhas: 78625 | Coluna:  5


## Mineração das regras

#### Enconder da base

In [14]:
def transactionEncoderData(dados:np.ndarray) -> pd.DataFrame:
    transEncoder = TransactionEncoder()
    a_data = transEncoder.fit(dados).transform(dados)
    dados = pd.DataFrame(a_data, columns=transEncoder.columns_)
    return dados*1

#### Itens frequentes
> <p style='text-align: justify;'><font size=2.85>Fpgrowth foi escolhido por ser mais eficiente</font></p>

In [15]:
def getFrequentItemsets(encoderData:pd.DataFrame, minsup:float=0.3, useColnames:bool=True) -> pd.DataFrame:
    return fpgrowth(encoderData, min_support=minsup, use_colnames=useColnames)

#### Regras

In [16]:
def getAssociationRules(itemsets:pd.DataFrame, metric:str='lift', minThreshold:int=1) -> pd.DataFrame:
    return association_rules(itemsets, metric=metric, min_threshold=minThreshold)

### Acidentes sem vítimas

In [17]:
sem_vitimas = transactionEncoderData(sem_vitimas.values)
frequent_itemsets = getFrequentItemsets(sem_vitimas)
rules_sem_vitimas = getAssociationRules(frequent_itemsets)
rules_sem_vitimas = rules_sem_vitimas.sort_values(by='lift', ascending=False)
rules_sem_vitimas[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.70').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,"(tempo bom, pequeno porte)",(reta),0.769513,1.083880,0.023370,1.258371
1,(tempo bom),(reta),0.760412,1.071061,0.030629,1.210572
2,"(tempo bom, dia)",(reta),0.759582,1.069891,0.021190,1.206391
3,(tempo bom),(dia),0.703396,1.057203,0.023107,1.128318
4,"(reta, tempo bom)",(dia),0.702628,1.056049,0.017216,1.125403


> <p style='text-align: justify;'><font size=2.85>Nos resultados da análise das regras para acidentes sem vítimas, tivemos uma regra com o maior valor de Lift de 1.08 e ela mostra que há, principalmente, acidentes envolvendo veículo de pequeno porte em tempo bom nas pistas com traçado reto.</font></p>

### Acidentes com vítimas feridas

In [18]:
vitimas_feridas = transactionEncoderData(vitimas_feridas.values)
frequent_itemsets = getFrequentItemsets(vitimas_feridas)
rules_vitimas_feridas = getAssociationRules(frequent_itemsets)
rules_vitimas_feridas = rules_vitimas_feridas.sort_values(by='lift', ascending=False)
rules_vitimas_feridas[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.60').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,(reta),(tempo bom),0.687916,1.050160,0.022328,1.105284
1,(tempo bom),(reta),0.713635,1.050160,0.022328,1.119029
2,(tempo bom),(dia),0.644279,1.047977,0.019321,1.082917
3,(dia),(tempo bom),0.686486,1.047977,0.019321,1.100243
4,(pequeno porte),(dia),0.615903,1.001820,0.000560,1.002913


> <p style='text-align: justify;'><font size=2.85>Para o cluster de vítimas feridas, temos a principal regra com um lift de 1.05 indicando que quando ocorrem colisões em pistas retas com tempo bom é provável que os envolvidos sofram algum tipo de ferimento. Também temos nesse grupo que os acidentes que causam ferimentos ocorrem durante o período do dia e também envolvem veículos de pequeno porte.</font></p>

### Acidentes com vítimas fatais

In [19]:
vitimas_fatais = transactionEncoderData(vitimas_fatais.values)
frequent_itemsets = getFrequentItemsets(vitimas_fatais)
rules_vitimas_fatais = getAssociationRules(frequent_itemsets)
rules_vitimas_fatais = rules_vitimas_fatais.sort_values(by='lift', ascending=False)
rules_vitimas_fatais[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.60').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,(noite),(reta),0.735494,1.054720,0.019965,1.144262
1,(dia),(tempo bom),0.696116,1.044407,0.014112,1.097398
2,(reta),(tempo bom),0.692548,1.039053,0.018151,1.084661
3,(tempo bom),(reta),0.724568,1.039053,0.018151,1.098873
4,(pequeno porte),(reta),0.710476,1.018843,0.005890,1.045385


> <p style='text-align: justify;'><font size=2.85>Em acidentes com vítimas fatais, a nossa principal regra, com lift de 1.05, nos mostra que nesse cluster os acidentes com mortes ocorrem, principalmente, durante a noite e em pistas com traçado reto. Boa parte também ocorre no período do dia e com um tempo bom.</font></p>